In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.models import Model
from keras.layers import (
    Dense,
    Input,
    Conv2D
)

from skimage.io import imread
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score

%matplotlib inline

In [ ]:
import sys
sys.path.append('..')

from utils.s3_class import S3Functions

s3_funcs = S3Functions(bucket_name='jdgallegoq-pinacle')

In [ ]:
import keras
import tensorflow as tf
print('keras version: ', keras.__version__)
print('tensorflow version: ', tf.__version__)

In [ ]:
# seeds to stop (try) random behaviour
seed = 42
rng = np.random.RandomState(seed)

In [ ]:
# data path
DATA_PATH = 'mnist/'

In [ ]:
train = pd.read_csv(s3_funcs.read_object(key=DATA_PATH+'train.csv'))
train.info()

In [ ]:
# read images
temp = []
for img_name in tqdm(train.filename):
    img = s3_funcs.read_image(key=DATA_PATH+'images/'+img_name)
    temp.append(img)

train_x = np.stack(temp)
train_x = train_x.reshape(len(train_x), -1, 784).astype(np.float32)